<img src="images/aaib.PNG" style="width:400px;height:250px;">

# In this notebook:

1. **Sentiment analysis** using bag of words
\
&nbsp;
2. **Topic modelling** using TF-IDF vectors

# Setup

In [18]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2


In [19]:
!pip install -U scikit-learn --upgrade

  Using cached scikit_learn-0.24.2-cp36-cp36m-win_amd64.whl (6.8 MB)


In [20]:
# Commonly used libraries
import numpy as np
import pandas as pd

# From ScikitLearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

# Sentiment analysis

Dataset: [Twitter US Airline Sentiment](https://https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment)

Sentiment can be negative, positive or neutral

In [8]:
data = pd.read_csv('datasets/Tweets.csv')
data.head(10)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,NaN,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,570299953286942721,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",NaN,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,570295459631263746,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [21]:
# We drop all columns but ID, sentiment, and text
data = data[['tweet_id', 'airline_sentiment', 'text']]
data.head(10)

,tweet_id,airline_sentiment,text
0,570306133677760513,neutral,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,@VirginAmerica and it's a really big bad thing...
5,570300767074181121,negative,@VirginAmerica seriously would pay $30 a fligh...
6,570300616901320704,positive,"@VirginAmerica yes, nearly every time I fly VX..."
7,570300248553349120,neutral,@VirginAmerica Really missed a prime opportuni...
8,570299953286942721,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,570295459631263746,positive,"@VirginAmerica it was amazing, and arrived an ..."


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
print(sklearn.__version__)

# Now we use a bag of word vectorizer that can turn a text into a vector
vectorizer = CountVectorizer()
vectorizer.fit(data.text)

# How many words do we have in our corpus?
print("Dimensions of the vectors: " + str(vectorizer.get_feature_names_out().size))

# Show a random word
print("\nDimension number 2000 represents the word: " + vectorizer.get_feature_names_out()[2000])

# Convert all texts into BoW representation
X = vectorizer.transform(data.text)
print("\n" + str(X.shape))

0.24.2


AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names_out'

In [ ]:
# Let's split the data to training and test (you already know this)
X_train, X_test, y_train, y_test = train_test_split(X, data.airline_sentiment, test_size = 0.2, random_state = 0)

# Let's train an SVM to do the sentiment analysis
clf_svm = svm.SVC(max_iter=1000, gamma='scale', kernel = "rbf", random_state=0)
clf_svm.fit(X_train, y_train)

In [ ]:
# Use the SVM to make predictions for the test set
y_pred = clf_svm.predict(X_test)

# Let's evaluate the performance
print("Confusion matrix")
print(clf_svm.classes_)
print(confusion_matrix(y_test, y_pred))

print("\nAccuracy")
print(accuracy_score(y_test, y_pred))

# Topic Modelling

We use the [20 newsgroups text dataset](https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset), a standard dataset provided by scikit consisting of 18.846 texts from 20 topics 

In [ ]:
data_train = fetch_20newsgroups(subset='train') 

In [ ]:
print("Topics: " + str(data_train.target_names))

print("\nData example:")
print(data_train.target_names[data_train.target[200]]) # we choose an unsupervised approach so we won't need them
print(data_train.data[200])

In [ ]:
# Tf-idf Vectorizer to turn the texts into Tf-idf vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_train.data)

In [ ]:
# What are the most important words in a document?
feature_array = np.array(vectorizer.get_feature_names_out())

tfidf_sorting = np.argsort(X[200].toarray()).flatten()[::-1]
n = 5
top_n = feature_array[tfidf_sorting][:n]
print(top_n)


In [ ]:
# Now cluster them

kmeans = KMeans(n_clusters=6, random_state=0, n_init="auto").fit(X)
